In [ ]:
import sqlite3
import ollama

def get_db_data(query):
    """Retrieves gene expression data from the database based on the query."""
    conn = sqlite3.connect('./db/gene_expression.db')
    cursor = conn.cursor()

    # Construct the SQL query based on the user's query
    # This is a simplified example, you'll need to refine it
    # based on the complexity of your queries
    if "over expressed" in query.lower():
        sql_query = """
            SELECT g.gene_id, ge.tpm
            FROM gene_expression ge
            JOIN conditions c ON ge.condition_id = c.condition_id
            JOIN experiments e ON c.experiment_id = e.experiment_id
            JOIN organisms o ON e.organism_id = o.organism_id
            JOIN genes g ON ge.gene_id = g.gene_id
            WHERE o.organism_name = ? AND c.condition_name LIKE ?
            ORDER BY ge.tpm DESC
            LIMIT 5
        """
        params = (query.split('/')[0], '%' + query.split('/')[1] + '%')
    else:
        sql_query = """
            SELECT g.gene_id, ge.tpm
            FROM gene_expression ge
            JOIN conditions c ON ge.condition_id = c.condition_id
            JOIN experiments e ON c.experiment_id = e.experiment_id
            JOIN organisms o ON e.organism_id = o.organism_id
            JOIN genes g ON ge.gene_id = g.gene_id
            WHERE o.organism_name = ? AND c.condition_name LIKE ?
            ORDER BY ge.tpm DESC
            LIMIT 5
        """
        params = (query.split('/')[0], '%' + query.split('/')[1] + '%')

    cursor.execute(sql_query, params)
    results = cursor.fetchall()
    conn.close()
    return results

def generate_prompt(results):
    """Generates a prompt for the language model based on the retrieved data."""
    if not results:
        return "No data found for the given query."

    prompt = "Based on the following gene expression data:\n"
    for gene_id, tpm in results:
        prompt += f"Gene: {gene_id}, TPM: {tpm}\n"
    prompt += "Summarize the key findings and potential implications."
    return prompt

def run_rag(query):
    """Runs the RAG pipeline."""
    results = get_db_data(query)
    prompt = generate_prompt(results)
    response = ollama.generate(model='llama3.2', prompt=prompt)
    return response

# Example Usage
query = "Homo sapiens/Disease A"
response = run_rag(query)
print(response)